# Semantic Ranker Explained

The following notebook describes how to:

1. Create a sample index consisting of fictitious hotel data
1. Configure semantic ranker on this index
1. Compare hybrid query with and without semantic ranker
1. Debug how semantic ranker interprets the top 50 results
1. Rerank specific documents given a query

## Prerequisites
* An Azure subscription, with access to Azure OpenAI.
* Azure AI Search basic or higher for this workload with [semantic ranker enabled](https://learn.microsoft.com/azure/search/semantic-how-to-enable-disable?tabs=enable-portal)
* A deployment of the text-embedding-3-large model on Azure OpenAI.

## Set up a Python virtual environment in Visual Studio Code
* Open the Command Palette (Ctrl+Shift+P).
* Search for Python: Create Environment.
* Select Venv.
* Select a Python interpreter. Choose 3.10 or later.

It can take a minute to set up. If you run into problems, see Python environments in VS Code.

### 1. Install packages

In [12]:
! pip install -r requirements.txt --quiet

### 2. Load .env file (Copy .env-sample to .env and update accordingly)

In [13]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
# You do not need a key if you are using keyless authentication
# To learn more, please visit https://learn.microsoft.com/azure/search/search-security-rbac
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY")) if len(os.getenv("AZURE_SEARCH_ADMIN_KEY", "")) > 0 else DefaultAzureCredential()
index_name = os.environ["AZURE_SEARCH_INDEX"]
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
# Data is pre-vectorized using text-embedding-3-large
model_name = "text-embedding-3-large"
aoai_embedding_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "text-embedding-3-large")
# You do not need a key if you are using keyless authentication
# To learn more, please visit https://learn.microsoft.com/azure/search/search-howto-managed-identities-data-sources and https://learn.microsoft.com/azure/developer/ai/keyless-connections
aoai_key = os.getenv("AZURE_OPENAI_KEY", None)

## 3. Create Search Index

Creates a search index in Azure AI Search to host the sample documents. The index name is specified through an environment variable. If an index already exists with that name, either delete it from your search service or provide a unique name.

In [14]:
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    ComplexField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters
)
from azure.search.documents.indexes import SearchIndexClient


index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="HotelId", type="Edm.String", key=True, hidden=False, filterable=True, sortable=False, facetable=False, searchable=True),
        SearchField(name="HotelName", type="Edm.String", hidden=False, filterable=False, sortable=False, facetable=False, searchable=True),
        SearchField(name="Description", type="Edm.String", hidden=False, filterable=False, sortable=False, facetable=False, searchable=True),
        SearchField(name="DescriptionEmbedding", type="Collection(Edm.Single)", hidden=False, searchable=True, vector_search_dimensions=3072, vector_search_profile_name="hnsw"),
        SearchField(name="Description_fr", type="Edm.String", hidden=False, filterable=False, sortable=False, facetable=False, searchable=True, analyzer_name="fr.microsoft"),
        SearchField(name="Description_fr_Embedding", type="Collection(Edm.Single)", hidden=False, searchable=True, vector_search_dimensions=3072, vector_search_profile_name="hnsw"),
        SearchField(name="Category", type="Edm.String", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
        SearchField(name="Tags", type="Collection(Edm.String)", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
        SearchField(name="ParkingIncluded", type="Edm.Boolean", hidden=False, filterable=True, sortable=False, facetable=True, searchable=False),
        SearchField(name="LastRenovationDate", type="Edm.DateTimeOffset", hidden=False, filterable=False, sortable=True, facetable=False, searchable=False),
        SearchField(name="Rating", type="Edm.Double", hidden=False, filterable=True, sortable=True, facetable=True, searchable=False),
        ComplexField(name="Address", collection=False, fields=[
            SearchField(name="StreetAddress", type="Edm.String", hidden=False, filterable=False, sortable=False, facetable=False, searchable=True),
            SearchField(name="City", type="Edm.String", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
            SearchField(name="StateProvince", type="Edm.String", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
            SearchField(name="PostalCode", type="Edm.String", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
            SearchField(name="Country", type="Edm.String", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True)
        ]),
        SearchField(name="Location", type="Edm.GeographyPoint", hidden=False, filterable=True, sortable=True, facetable=False, searchable=False),
        ComplexField(name="Rooms", collection=True, fields=[
            SearchField(name="Description", type="Edm.String", hidden=False, filterable=False, sortable=False, facetable=False, searchable=True),
            SearchField(name="Description_fr", type="Edm.String", hidden=False, filterable=False, sortable=False, facetable=False, searchable=True, analyzer_name="fr.microsoft"),
            SearchField(name="Type", type="Edm.String", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
            SearchField(name="BaseRate", type="Edm.Double", hidden=False, filterable=True, sortable=False, facetable=True, searchable=False),
            SearchField(name="BedOptions", type="Edm.String", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
            SearchField(name="SleepsCount", type="Edm.Int64", hidden=False, filterable=True, sortable=False, facetable=True, searchable=False),
            SearchField(name="SmokingAllowed", type="Edm.Boolean", hidden=False, filterable=True, sortable=False, facetable=True, searchable=False),
            SearchField(name="Tags", type="Collection(Edm.String)", hidden=False, filterable=True, sortable=False, facetable=True, searchable=True),
        ])
    ],
    vector_search=VectorSearch(
        profiles=[VectorSearchProfile(name="hnsw", vectorizer_name="openai", algorithm_configuration_name="hnsw")],
        algorithms=[HnswAlgorithmConfiguration(name="hnsw")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="openai",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=aoai_endpoint,
                    deployment_name=aoai_embedding_deployment,
                    model_name=model_name,
                    api_key=aoai_key,
                )
            )
        ]
    )
)

index_client = SearchIndexClient(endpoint, credential)
result = index_client.create_or_update_index(index)
print("Created sample index")

Created sample index


## 4. Upload sample documents

Upload the sample hotel documents to the index


In [15]:
from azure.search.documents import SearchClient
import json

client = SearchClient(endpoint, index_name, credential)
with open("../../../data/hotels.json", encoding="utf-8", mode="r") as f:
    documents = json.load(f)
    client.upload_documents(documents)
    print("Uploaded sample documents")

Uploaded sample documents


## 5. Setup Semantic Ranker Configuration

Usage of semantic ranker requires a [configuration](https://learn.microsoft.com/en-us/azure/search/semantic-how-to-configure?tabs=portal). A configuration

+ Lists the fields semantic ranker uses to rerank documents
+ You can create many semantic configurations
+ Semantic configurations may have a single title field, multiple content fields, and multiple keyword fields
+ You can also set a default semantic configuration to be used on queries that have semantic ranker enabled
+ Semantic configurations can always be set on individual queries

In [16]:
from azure.search.documents.indexes.models import SemanticSearch, SemanticConfiguration, SemanticField, SemanticPrioritizedFields

index.semantic_search = SemanticSearch(
    default_configuration_name="semantic-config",
    configurations=[
        SemanticConfiguration(
            name="semantic-config",
            prioritized_fields=SemanticPrioritizedFields(
                content_fields=[SemanticField(field_name="Description")]
            )
        ),
        SemanticConfiguration(
            name="semantic-config-with-title",
            prioritized_fields=SemanticPrioritizedFields(
                title_field=SemanticField(field_name="HotelName"),
                content_fields=[SemanticField(field_name="Description")]
            )
        ),
        SemanticConfiguration(
            name="semantic-config-with-title-and-keywords",
            prioritized_fields=SemanticPrioritizedFields(
                title_field=SemanticField(field_name="HotelName"),
                content_fields=[SemanticField(field_name="Description")],
                keywords_fields=[SemanticField(field_name="Tags")]
            )
        ),
        SemanticConfiguration(
            name="semantic-config-with-title-and-keywords-extended",
            prioritized_fields=SemanticPrioritizedFields(
                title_field=SemanticField(field_name="HotelName"),
                content_fields=[SemanticField(field_name="Description")],
                keywords_fields=[SemanticField(field_name="Tags"), SemanticField(field_name="Address/City"), SemanticField(field_name="Address/Country")]
            )
        )
    ]
)

result = index_client.create_or_update_index(index)

print("Updated index with semantic configuration")

Updated index with semantic configuration


### 6. Run Semantic Ranker Queries

The following function allows you to run semantic ranker queries and compare results. In the remaining cells we'll see how to use the various semantic ranker options

In [17]:
from azure.search.documents.models import VectorizableTextQuery
from IPython.display import display, Markdown
from typing import List,Optional

import pandas as pd


def run_query(query: str, semantic_config: Optional[str] = None, include_debug: bool = False, include_captions: bool = False, include_answers: bool = False, document_ids: Optional[List[str]] = None) -> None:
    """
    Executes a query and returns the results as a pandas DataFrame.

    Args:
        query (str): The search query to execute.
        semantic_config (Optional[str], optional): Configuration for semantic search. Defaults to None.
        include_debug (bool, optional): Whether to include debug information in the results. Defaults to False.
        include_captions (bool, optional): Whether to include captions in the results. Defaults to False.
        include_answers (bool, optional): Whether to include extracted answers in the results. Defaults to False.
        document_ids (Optional[List[str]], optional): A list of document IDs to filter results. Defaults to None.

    Displays the output results for comparison in the notebook
    """
    
    document_id_filter = None
    if document_ids:
        document_id_filter = " or ".join(f"HotelId eq '{document_id}'" for document_id in document_ids)
    results = client.search(
        search_text=query,
        vector_queries=[VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="DescriptionEmbedding")],
        select="HotelId,HotelName,Description",
        top=50,
        query_type="semantic",
        filter=document_id_filter,
        semantic_configuration_name=semantic_config,
        debug="semantic",
        query_answer="extractive|count-3",
        query_caption="extractive|highlight-true"
    )
    answers = [answer.as_dict() for answer in results.get_answers()]
    results_with_debug_info = []
    for result in list(results):
        debug_info = result["@search.document_debug_info"].as_dict()
        captions = result["@search.captions"]
        if len(captions) > 0:
            first_caption = captions[0].as_dict()
            result["Caption Highlights"] = first_caption['highlights']
        results_with_debug_info.append({
            "@search.score": result["@search.score"],
            "@search.reranker_score": result["@search.reranker_score"],
            "HotelId": result["HotelId"],
            "HotelName": result["HotelName"],
            "Description": result["Description"],
            "Caption Highlights": result["Caption Highlights"],
            "Title": debug_info["semantic"]["reranker_input"].get("title"),
            "Content": debug_info["semantic"]["reranker_input"].get("content"),
            "Keywords": debug_info["semantic"]["reranker_input"].get("keywords")
        })
    
    columns = ["@search.score", "@search.reranker_score", "HotelId", "HotelName", "Description"]
    if include_captions:
        columns.extend(["Caption Highlights"])
    debug_columns = ["Title", "Content", "Keywords"]
    if include_debug:
        columns.extend(debug_columns)
    df = pd.DataFrame(results_with_debug_info, columns=columns)
    df_no_reranker = df.sort_values(by="@search.score", ascending=False)
    if include_debug:
        df_no_reranker = df_no_reranker.drop(columns=debug_columns)
    df = pd.DataFrame({
        "Reranked Index": df.index.map({reranked_idx: i for i, reranked_idx in enumerate(df.index)}),
        "Original Index": df.index.map({original_idx: i for i, original_idx in enumerate(df_no_reranker.index)}),
        **df.to_dict('list')
    })

    def highlight_rerank_row_changes(row: pd.Series):
        reranked_index = row.iloc[0]
        original_index = row.iloc[1]
        first_column_style = ''
        if original_index > reranked_index:
            first_column_style = 'background-color: green'
        elif original_index < reranked_index:
            first_column_style =  'background-color: red'

        return [first_column_style, first_column_style] + ['' for i in range(len(row) - 2)]
    df_style = {
        'max-width': '500px',
        'text-align': 'left',
        'white-space': 'normal',
        'word-wrap': 'break-word'
    }

    reranked_df = df.head(5)
    reranked_df.columns.name = "Reranked Index"
    reranked_df = reranked_df.style.apply(highlight_rerank_row_changes, axis=1).set_properties(**df_style).hide(axis="index")
    display(Markdown("### Reranked Results"))
    display(reranked_df)

    no_rerank_df = pd.DataFrame({
        "Reranked Index": df_no_reranker.index.map({reranked_idx: i for i, reranked_idx in enumerate(df.index)}),
        "Original Index": df_no_reranker.index.map({original_idx: i for i, original_idx in enumerate(df_no_reranker.index)}),
        **df_no_reranker.to_dict('list')
    })
    no_rerank_df = no_rerank_df.reset_index(drop=True)
    no_rerank_df.columns.name = "Original Index"
    no_rerank_df = no_rerank_df.head(5).style.apply(highlight_rerank_row_changes, axis=1).set_properties(**df_style).hide(axis="index")

    if not document_ids:
        display(Markdown("### Original Results"))
        display(no_rerank_df)

    if include_answers:
        display(Markdown("### Answers"))
        display(pd.DataFrame(answers, columns=["key", "score", "text"]).style.set_properties(**df_style).hide(axis="index"))



### 6.1 Running a semantic query

Semantic ranker takes results output from a plain text, vector, or hybrid search and re-ranks them

+ Only the top 50 documents are reranked
+ Reranking comparisons are made on the basis of how well the given document matches the query. The following criteria is used:

| Score | Meaning |
| -- | --
| 4.0 | The document is highly relevant and answers the question completely, though the passage might contain extra text unrelated to the question. |
| 3.0 | The document is relevant but lacks details that would make it complete. |
| 2.0 | The document is somewhat relevant; it answers the question either partially or only addresses some aspects of the question. |
| 1.0 | The document is related to the question, and it answers a small part of it. |
| 0.0 | The document is irrelevant. |


The `run_query` function in this example outputs 2 tables:

1. Reranked results, showcasing what is returned by a hybrid search (text + vector merged using [Reciprocal Rank Fusion](https://learn.microsoft.com/azure/search/hybrid-search-ranking)) and then re-ranked using semantic ranker
   1. Green rows represent documents that semantic ranker thinks are a better fit for the query than the original hybrid search
   1. Red rows represent documents that semantic ranker thinks are a worse fit for the query than the original hybrid search
1. Original results, showcasing what is returnd by the original hybrid search without semantic ranker input.
   1. The same coloring scheme is used, but you can now see what the original results from the top 50 were.

In [19]:
run_query("new york hotel with pool or gym", semantic_config="semantic-config-with-title-and-keywords")


### Reranked Results

Reranked Index,Original Index,@search.score,@search.reranker_score,HotelId,HotelName,Description
0,2,0.032002,3.015908,6,King's Cellar Hotel,"Newest kid on the downtown block. Steps away from the most popular destinations in downtown, enjoy free WiFi, an indoor rooftop pool & fitness center, 24 Grab'n'Go & drinks at the bar"
1,0,0.032796,2.821660,36,Hotel on the Harbor,"Stunning Downtown Hotel with indoor Pool. Ideally located close to theatres, museums and the convention center. Indoor Pool and Sauna and fitness centre. Popular Bar & Restaurant"
2,4,0.031498,2.821660,16,Double Sanctuary Resort,"5 star Luxury Hotel - Biggest Rooms in the city. #1 Hotel in the area listed by Traveler magazine. Free WiFi, Flexible check in/out, Fitness Center & espresso in room."
3,6,0.030077,2.651160,12,Winter Panorama Resort,"Plenty of great skiing, outdoor ice skating, sleigh rides, tubing and snow biking. Yoga, group exercise classes and outdoor hockey are available year-round, plus numerous options for shopping as well as great spa services. Newly-renovated with large rooms, free 24-hr airport shuttle & a new restaurant. Rooms/suites offer mini-fridges & 49-inch HDTVs."
4,11,0.027106,2.622093,28,City Center Summer Wind Resort,"Eco-friendly from our gardens to table, with a rooftop serenity pool and outdoor seating to take in the sunset. Just steps away from the Convention Center. Located in the heart of downtown with modern rooms with stunning city views, 24-7 dining options, free WiFi and easy valet parking."


### Original Results

Reranked Index,Original Index,@search.score,@search.reranker_score,HotelId,HotelName,Description
1,0,0.032796,2.821660,36,Hotel on the Harbor,"Stunning Downtown Hotel with indoor Pool. Ideally located close to theatres, museums and the convention center. Indoor Pool and Sauna and fitness centre. Popular Bar & Restaurant"
14,1,0.032292,2.361027,1,Stay-Kay City Hotel,"This classic hotel is fully-refurbished and ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Times Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities."
0,2,0.032002,3.015908,6,King's Cellar Hotel,"Newest kid on the downtown block. Steps away from the most popular destinations in downtown, enjoy free WiFi, an indoor rooftop pool & fitness center, 24 Grab'n'Go & drinks at the bar"
23,3,0.031545,2.123001,17,City Skyline Antiquity Hotel,"In vogue since 1888, the Antiquity Hotel takes you back to bygone era. From the crystal chandeliers that adorn the Green Room, to the arched ceilings of the Grand Hall, the elegance of old New York beckons. Elevate Your Experience. Upgrade to a premiere city skyline view for less, where old world charm combines with dramatic views of the city, local cathedral and midtown."
2,4,0.031498,2.821660,16,Double Sanctuary Resort,"5 star Luxury Hotel - Biggest Rooms in the city. #1 Hotel in the area listed by Traveler magazine. Free WiFi, Flexible check in/out, Fitness Center & espresso in room."


### 6.2 - Debugging Semantic Ranker

Semantic ranker has a limited amount of content from every document that it can accept. This content are split across the title, content, and keywords fields defined in the semantic configuration. When dealing with unexpected results from semantic ranker, it can be useful to see the actual input. Adding `include_debug=True` to the `run_query` function adds the `Title`, `Content`, and `Keywords` columns to the output tables, allowing examination of what input was actually sent to semantic ranker.

In [9]:
run_query("new york hotel with pool or gym", semantic_config="semantic-config-with-title-and-keywords", include_debug=True)


### Reranked Results

Reranked Index,Original Index,@search.score,@search.reranker_score,HotelId,HotelName,Description,Title,Content,Keywords
0,2,0.032002,3.015908,6,King's Cellar Hotel,"Newest kid on the downtown block. Steps away from the most popular destinations in downtown, enjoy free WiFi, an indoor rooftop pool & fitness center, 24 Grab'n'Go & drinks at the bar",King's Cellar Hotel,"Newest kid on the downtown block. Steps away from the most popular destinations in downtown, enjoy free WiFi, an indoor rooftop pool & fitness center, 24 Grab'n'Go & drinks at the bar",free wifi pool bar
1,0,0.032796,2.821660,36,Hotel on the Harbor,"Stunning Downtown Hotel with indoor Pool. Ideally located close to theatres, museums and the convention center. Indoor Pool and Sauna and fitness centre. Popular Bar & Restaurant",Hotel on the Harbor,"Stunning Downtown Hotel with indoor Pool. Ideally located close to theatres, museums and the convention center. Indoor Pool and Sauna and fitness centre. Popular Bar & Restaurant",bar pool 24-hour front desk service
2,4,0.031498,2.821660,16,Double Sanctuary Resort,"5 star Luxury Hotel - Biggest Rooms in the city. #1 Hotel in the area listed by Traveler magazine. Free WiFi, Flexible check in/out, Fitness Center & espresso in room.",Double Sanctuary Resort,"5 star Luxury Hotel - Biggest Rooms in the city. #1 Hotel in the area listed by Traveler magazine. Free WiFi, Flexible check in/out, Fitness Center & espresso in room.",view pool restaurant bar continental breakfast
3,6,0.030077,2.651160,12,Winter Panorama Resort,"Plenty of great skiing, outdoor ice skating, sleigh rides, tubing and snow biking. Yoga, group exercise classes and outdoor hockey are available year-round, plus numerous options for shopping as well as great spa services. Newly-renovated with large rooms, free 24-hr airport shuttle & a new restaurant. Rooms/suites offer mini-fridges & 49-inch HDTVs.",Winter Panorama Resort,"Plenty of great skiing, outdoor ice skating, sleigh rides, tubing and snow biking. Yoga, group exercise classes and outdoor hockey are available year-round, plus numerous options for shopping as well as great spa services. Newly-renovated with large rooms, free 24-hr airport shuttle & a new restaurant. Rooms/suites offer mini-fridges & 49-inch HDTVs.",restaurant bar pool
4,11,0.027106,2.622093,28,City Center Summer Wind Resort,"Eco-friendly from our gardens to table, with a rooftop serenity pool and outdoor seating to take in the sunset. Just steps away from the Convention Center. Located in the heart of downtown with modern rooms with stunning city views, 24-7 dining options, free WiFi and easy valet parking.",City Center Summer Wind Resort,"Eco-friendly from our gardens to table, with a rooftop serenity pool and outdoor seating to take in the sunset. Just steps away from the Convention Center. Located in the heart of downtown with modern rooms with stunning city views, 24-7 dining options, free WiFi and easy valet parking.",restaurant view concierge


### Original Results

Reranked Index,Original Index,@search.score,@search.reranker_score,HotelId,HotelName,Description
1,0,0.032796,2.821660,36,Hotel on the Harbor,"Stunning Downtown Hotel with indoor Pool. Ideally located close to theatres, museums and the convention center. Indoor Pool and Sauna and fitness centre. Popular Bar & Restaurant"
14,1,0.032292,2.361027,1,Stay-Kay City Hotel,"This classic hotel is fully-refurbished and ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Times Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities."
0,2,0.032002,3.015908,6,King's Cellar Hotel,"Newest kid on the downtown block. Steps away from the most popular destinations in downtown, enjoy free WiFi, an indoor rooftop pool & fitness center, 24 Grab'n'Go & drinks at the bar"
23,3,0.031545,2.123001,17,City Skyline Antiquity Hotel,"In vogue since 1888, the Antiquity Hotel takes you back to bygone era. From the crystal chandeliers that adorn the Green Room, to the arched ceilings of the Grand Hall, the elegance of old New York beckons. Elevate Your Experience. Upgrade to a premiere city skyline view for less, where old world charm combines with dramatic views of the city, local cathedral and midtown."
2,4,0.031498,2.821660,16,Double Sanctuary Resort,"5 star Luxury Hotel - Biggest Rooms in the city. #1 Hotel in the area listed by Traveler magazine. Free WiFi, Flexible check in/out, Fitness Center & espresso in room."


### 6.3 - Semantic Answers And Captions

Semantic ranker also supports automatically returning [answers and captions](https://learn.microsoft.com/azure/search/semantic-answers) from documents returned for a query.

1. Semantic ranker attempts to interpret the query as a natural language question and find the best answer from the top 50 documents that fits it. Unlike [Retrieval Augmented Generation (RAG)](https://learn.microsoft.com/azure/search/retrieval-augmented-generation-overview), this answer is extractive only
1. Semantic ranker attempts to extract a representative caption, or snippet of text, from the document that is relevant to the query. This caption also has highlights that can be displayed in a search UI to help guide users to the most relevant results.

Adding `include_captions=True`, `include_answers=True` to the query adds a `Caption Highlights` column to the `Reranked Results` table and an additonal `Answers` table to review the semantic answers

In [10]:
run_query("best hotel?", semantic_config="semantic-config-with-title-and-keywords", include_captions=True, include_answers=True)


### Reranked Results

Reranked Index,Original Index,@search.score,@search.reranker_score,HotelId,HotelName,Description,Caption Highlights
0,0,0.033333,2.907087,16,Double Sanctuary Resort,"5 star Luxury Hotel - Biggest Rooms in the city. #1 Hotel in the area listed by Traveler magazine. Free WiFi, Flexible check in/out, Fitness Center & espresso in room.","5 star Luxury Hotel - Biggest Rooms in the city. #1 Hotel in the area listed by Traveler magazine. Free WiFi, Flexible check in/out, Fitness Center & espresso in room."
1,2,0.028898,2.605079,50,Head Wind Resort,"The best of old town hospitality combined with views of the river and cool breezes off the prairie. Our penthouse suites offer views for miles and the rooftop plaza is open to all guests from sunset to 10 p.m. Enjoy a complimentary continental breakfast in the lobby, and free Wi-Fi throughout the hotel.","The best of old town hospitality combined with views of the river and cool breezes off the prairie. Our penthouse suites offer views for miles and the rooftop plaza is open to all guests from sunset to 10 p.m. Enjoy a complimentary continental breakfast in the lobby, and free Wi-Fi throughout the hotel."
2,3,0.028298,2.575658,18,Ocean Water Resort & Spa,"New Luxury Hotel for the vacation of a lifetime. Bay views from every room, location near the pier, rooftop pool, waterfront dining & more.","New Luxury Hotel for the vacation of a lifetime. Bay views from every room, location near the pier, rooftop pool, waterfront dining & more."
3,1,0.032522,2.559352,13,Luxury Lion Resort,"Unmatched Luxury. Visit our downtown hotel to indulge in luxury accommodations. Moments from the stadium and transportation hubs, we feature the best in convenience and comfort.","Unmatched Luxury. Visit our downtown hotel to indulge in luxury accommodations. Moments from the stadium and transportation hubs, we feature the best in convenience and comfort."
4,12,0.027584,2.534894,36,Hotel on the Harbor,"Stunning Downtown Hotel with indoor Pool. Ideally located close to theatres, museums and the convention center. Indoor Pool and Sauna and fitness centre. Popular Bar & Restaurant","Stunning Downtown Hotel with indoor Pool. Ideally located close to theatres, museums and the convention center. Indoor Pool and Sauna and fitness centre. Popular Bar & Restaurant."


### Original Results

Reranked Index,Original Index,@search.score,@search.reranker_score,HotelId,HotelName,Description,Caption Highlights
0,0,0.033333,2.907087,16,Double Sanctuary Resort,"5 star Luxury Hotel - Biggest Rooms in the city. #1 Hotel in the area listed by Traveler magazine. Free WiFi, Flexible check in/out, Fitness Center & espresso in room.","5 star Luxury Hotel - Biggest Rooms in the city. #1 Hotel in the area listed by Traveler magazine. Free WiFi, Flexible check in/out, Fitness Center & espresso in room."
3,1,0.032522,2.559352,13,Luxury Lion Resort,"Unmatched Luxury. Visit our downtown hotel to indulge in luxury accommodations. Moments from the stadium and transportation hubs, we feature the best in convenience and comfort.","Unmatched Luxury. Visit our downtown hotel to indulge in luxury accommodations. Moments from the stadium and transportation hubs, we feature the best in convenience and comfort."
1,2,0.028898,2.605079,50,Head Wind Resort,"The best of old town hospitality combined with views of the river and cool breezes off the prairie. Our penthouse suites offer views for miles and the rooftop plaza is open to all guests from sunset to 10 p.m. Enjoy a complimentary continental breakfast in the lobby, and free Wi-Fi throughout the hotel.","The best of old town hospitality combined with views of the river and cool breezes off the prairie. Our penthouse suites offer views for miles and the rooftop plaza is open to all guests from sunset to 10 p.m. Enjoy a complimentary continental breakfast in the lobby, and free Wi-Fi throughout the hotel."
2,3,0.028298,2.575658,18,Ocean Water Resort & Spa,"New Luxury Hotel for the vacation of a lifetime. Bay views from every room, location near the pier, rooftop pool, waterfront dining & more.","New Luxury Hotel for the vacation of a lifetime. Bay views from every room, location near the pier, rooftop pool, waterfront dining & more."
27,4,0.028283,2.213390,35,Bellevue Suites,"Comfortable city living in the very center of downtown Bellevue. Newly reimagined, this hotel features apartment-style suites with sleeping, living and work spaces. Located across the street from the Light Rail to downtown. Free shuttle to the airport.","Comfortable city living in the very center of downtown Bellevue. Newly reimagined, this hotel features apartment-style suites with sleeping, living and work spaces. Located across the street from the Light Rail to downtown. Free shuttle to the airport."


### Answers

key,score,text
16,0.985000,"5 star Luxury Hotel - Biggest Rooms in the city. #1 Hotel in the area listed by Traveler magazine. Free WiFi, Flexible check in/out, Fitness Center & espresso in room."
13,0.945000,"Unmatched Luxury. Visit our downtown hotel to indulge in luxury accommodations. Moments from the stadium and transportation hubs, we feature the best in convenience and comfort."
27,0.939000,"Complimentary Airport Shuttle & WiFi. Book Now and save - Spacious All Suite Hotel, Indoor Outdoor Pool, Fitness Center, Florida Green certified, Complimentary Coffee, HDTV."


### 6.4 - Semantic Ranker on Individual Documents

Sometimes it can be useful to find the semantic ranker score for a specific document given a query. By supplying the right [filter](https://learn.microsoft.com/azure/search/search-filters) to your search, you can extract the reranker score for any given document and query, provided you know that document's ID. Adding the `document_ids=[id1, id2, ... idN]` parameter to `run_query` allows you to extract this ranker score for the appropriate document. Please note that only up to 50 documents can be reranked for a request, so requests to rerank more than 50 documents at a time must be batched.

In [11]:
run_query("best hotel?", semantic_config="semantic-config-with-title-and-keywords", document_ids=["18"], include_debug=True)

### Reranked Results

Reranked Index,Original Index,@search.score,@search.reranker_score,HotelId,HotelName,Description,Title,Content,Keywords
0,0,0.033333,2.575658,18,Ocean Water Resort & Spa,"New Luxury Hotel for the vacation of a lifetime. Bay views from every room, location near the pier, rooftop pool, waterfront dining & more.",Ocean Water Resort & Spa,"New Luxury Hotel for the vacation of a lifetime. Bay views from every room, location near the pier, rooftop pool, waterfront dining & more.",view pool restaurant


In [11]:
run_query("best hotel?", semantic_config="semantic-config-with-title-and-keywords", document_ids=["18", "6"], include_debug=True)

### Reranked Results

Reranked Index,Original Index,@search.score,@search.reranker_score,HotelId,HotelName,Description,Title,Content,Keywords
0,0,0.033060,2.575658,18,Ocean Water Resort & Spa,"New Luxury Hotel for the vacation of a lifetime. Bay views from every room, location near the pier, rooftop pool, waterfront dining & more.",Ocean Water Resort & Spa,"New Luxury Hotel for the vacation of a lifetime. Bay views from every room, location near the pier, rooftop pool, waterfront dining & more.",view pool restaurant
1,1,0.033060,2.066552,6,King's Cellar Hotel,"Newest kid on the downtown block. Steps away from the most popular destinations in downtown, enjoy free WiFi, an indoor rooftop pool & fitness center, 24 Grab'n'Go & drinks at the bar",King's Cellar Hotel,"Newest kid on the downtown block. Steps away from the most popular destinations in downtown, enjoy free WiFi, an indoor rooftop pool & fitness center, 24 Grab'n'Go & drinks at the bar",free wifi pool bar
